<a href="https://colab.research.google.com/github/statistics-jun/2022-1-ESAA/blob/main/ESAA_0502_20newsgroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2022-1 ESAA 5기 김혜준 실습 필사 - 0502(월) 9주차 과제
 
## <파이썬 머신러닝 완벽 가이드>
## Chapter 8 텍스트 분석

### 04 텍스트 분류 실습 - 20 뉴스그룹 분류
* **텍스트 분류** : 문서가 특정 분류 또는 카테고리에 속하는 것을 예측하는 기법 - 지도학습 적용(학습 모델 생성하여 이를 기반으로 다른 문서의 분류 예측)
  
  1. 텍스트 정규화
  2. 피처 벡터화 적용

    -> 희소 행렬 형태
    
    -> 로지스틱 회귀, 선형 서포트 벡터 머신, 나이브 베이즈 등의 알고리즘을 이용해 분류 효과적으로 처리
  3. 적합한 ML 알고리즘으로 분류를 학습/예측/평가
* 예제 데이터 : fetch_20newsgroups()

#### **텍스트 정규화**

데이터 불러오기

In [ ]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [ ]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* filnames : fetch_20newsgroups()가 데이터를 인터넷에서 내려받아 로컬 컴퓨터에 저장하는 디렉터리와 파일명

Target 클래스 구성 확인

In [ ]:
import pandas as pd

print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 뉴스그룹 기사 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등 다양한 정보 포함
  
  -> remove 파라미터를 이용하여 내용을 제외한 다른 정보 제거

subset 파라미터를 이용하여 학습/테스트 데이터 추출

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# subset='train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),
                   random_state=156)
X_train = train_news.data
y_train = train_news.target

# subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                   random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data), len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


#### **피처 벡터화 변환과 머신러닝 모델 학습/예측/평가**

카운트 기반 벡터화 - CountVectorizer

각 문서에서 해당 단어가 나타나는 횟수(Count)를 부여

* 테스트 데이터의 피처 벡터화 시 유의할 점
  1. 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체에 transform()을 적용하여 테스트 데이터를 변환
  
    -> 학습 시 설정된 CountVectorizer의 피처 개수와 테스트 데이터의 피처 개수를 같게 하기 위함
  2. fit_transform() 사용 불가

    -> 학습 시 사용된 피처 개수와 예측 시 사용할 피처 개수가 달라지기 때문

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 피처 벡터화 변환 수행
cnt_vext = CountVectorizer()
cnt_vext.fit(X_train)
X_train_cnt_vect = cnt_vext.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizor를 이용해 테스트 데이터를 피처 벡터화 변환 수행
X_test_cnt_vect = cnt_vext.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


로지스틱 회귀를 적용하여 뉴스그룹에 대한 분류 예측

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


* CountVectorized Logistic Regression 예측 정확도 : 0.608

TF-IDF 기반 벡터화 - TfidfVectorizer

개별 문서에서 자주 나타나는 단어에 높은 가중치를 주되, 모든 문서에서 전반적으로 자주 나타나는 단어에 대해서는 패널티를 주는 방식으로 값을 부여

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vext = TfidfVectorizer()
tfidf_vext.fit(X_train)
X_train_tfidf_vect = tfidf_vext.transform(X_train)
X_test_tfidf_vect = tfidf_vext.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

TF-IDF Logistic Regression의 예측 정확도는 0.674


* TF-IDF Logistic Regression 예측 정확도 : 0.674
* 예측 성능 비교 : TF-IDF 기반 > Count 기반
  - 문서 내 텍스트가 많고, 문서의 개수가 많은 경우 TF-IDF 벡터화가 더 좋은 예측 결과 도출

* 텍스트 분석에서 머신러닝 모델 성능 향상시키는 방법
  1. 최적의 ML 알고리즘 선택
  2. 최상의 피처 전처리 수행 - 텍스트 정규화, Count/TF-IDF 기반 피처 벡터화

TF-IDF 기반 벡터화에 다양한 파라미터 적용하여 예측 성능 측정

- max_df : 전체 문서에 걸쳐서 너무 높은 빈도수를 가지는 단어 피처를 제외
- stop_words : 특정 언어의 스톱워드로 지정단 단어는 추출에서 제외
- n_gram_range : BOW 모델의 단어 순서를 어느 정도 보강하기 위한 n_gram 범위를 설정, 튜플 형태로 (범위 최솟값, 범위 최댓값)을 지정

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1, 1)에서 (1, 2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


* TF-IDF Vectorized Logistic Regression 예측 정확도 : 0.692

GridSearchCV 기반 로지스틱 회귀 하이퍼 파라미터 최적화

* C : 규제 강도 조절 파라미터

In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C 값 도출 튜닝 수행. CV는 3 폴드 세트로 설정 
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf ,param_grid=params, cv=3, scoring='accuracy', verbose=1 )
grid_cv_lr.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가 
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.701


* Logistic Regression 최적 C 값 : 10
* TF-IDF Vectorized Logistic Regression 예측 정확도 : 0.701

#### **사이킷런 파이프라인(Pipeline) 사용 및 GridSearchCV와의 결합**

* Pipeline : 데이터의 가공, 변환 등의 전처리와 ML 알고리즘 적용을 한꺼번에 스트림 기반으로 처리

  => 모든 데이터 전처리 + Estimator

  - 직관적인 ML 모델 코드 생성
  - 수행 시간 절약

In [ ]:
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
                     ('lr_clf', LogisticRegression(random_state=156))])

Pipeline을 사용하여 텍스트 분류 수행

In [ ]:
from sklearn.pipeline import Pipeline

# TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidfVectorizer객체의 fit(), transform( )과 LogisticRegression의 fit(), predict( )가 필요 없음 
# pipeline의 fit( ) 과 predict( )만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.701


* Pipeline을 통한 Logistic Regression 예측 정확도 : 0.701

Pipeline 기반 GridSearchCV를 이용해 피처 벡터화를 위한 파라미터와 ML 알고리즘의 하이퍼 파라미터 한 번에 최적화

* Pipeline 기반 GridSearchCV에서 param_grid 입력 값 설정
  - 딕셔너리 형태의 Key와 Value 값, Value를 리스트 형태로 입력하는 것은 동일
  - 개별 객체 명과 파라미터명/하이퍼 파라미터명을 언더바 2개로 결합해 Key 값으로 할당

    예) tfidf_vect__ngram_range
* Pipeline + GridSearchCV 적용 시 유의할 점
  - 많은 튜닝 시간 소모


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될 
# 파라미터/하이퍼 파라미터 이름과 값을 설정. . 
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3 , scoring='accuracy',verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7536687914006531
Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.701


* Logistic Regression 최적 C 값 : 10
* TfidfVectorizer 최적 파라미터
  - max_df : 300
  - ngram_range : (1, 2)
* Pipeline을 통한 Logistic Regression 예측 정확도 : 0.701